In [ ]:
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt

# --- 1. O LABORATÓRIO (Uma imagem "fake" minúscula) ---
# Imagine uma imagem 4x4.
# Vamos definir uma MÁSCARA (Região Omega) onde queremos colar algo.
# 1 = Dentro da região de colagem (Incógnita)
# 0 = Fora (Borda fixa / Condição de Contorno)

mask = np.array([
    [0, 0, 0, 0],
    [0, 1, 1, 0],
    [0, 1, 1, 0],
    [0, 0, 0, 0]
])

print("Máscara (Região de Colagem):")
print(mask)

# --- 2. O MAPEAMENTO (Mapping) ---
# Precisamos numerar os pixels que são '1' na máscara.
# Ex: O pixel (1,1) será a incógnita 0. O pixel (1,2) será a incógnita 1...

# Encontra as coordenadas (y, x) onde mask == 1
coords = np.argwhere(mask == 1)
n_pixels = len(coords)

print(f"\nNúmero de incógnitas (pixels para resolver): {n_pixels}")

# Criar um dicionário para busca rápida: map[(y,x)] -> k (índice no sistema linear)
pixel_map = {}
for k, (y, x) in enumerate(coords):
    pixel_map[(y, x)] = k
    print(f"Pixel na coord ({y}, {x}) -> Variável x_{k} no sistema linear")

# --- 3. CONSTRUÇÃO DA MATRIZ LAPLACIANA (A) ---
# Vamos usar LIL matrix (Linked List) pois é eficiente para construir passo a passo
A = sp.lil_matrix((n_pixels, n_pixels))

# Para cada pixel k dentro da máscara...
for k, (y, x) in enumerate(coords):
    # 1. Diagonal Principal: O próprio pixel ganha peso 4
    A[k, k] = 4

    # 2. Vizinhos (Cima, Baixo, Esq, Dir)
    vizinhos = [
        (y-1, x), # Cima
        (y+1, x), # Baixo
        (y, x-1), # Esquerda
        (y, x+1)  # Direita
    ]

    for vy, vx in vizinhos:
        # Verificamos se o vizinho TAMBÉM é uma incógnita (está na máscara)
        if (vy, vx) in pixel_map:
            viz_idx = pixel_map[(vy, vx)]
            # Se é vizinho e é incógnita, ganha peso -1 na matriz A
            A[k, viz_idx] = -1
        else:
            # Se o vizinho NÃO está no map, ele é BORDA (Condição de Contorno).
            # Ele não entra na matriz A (lado esquerdo), ele vai para o vetor b (lado direito).
            # (No código final de imagem real, aqui somaríamos o valor do pixel no vetor b)
            pass

# --- 4. VISUALIZAÇÃO DA MATRIZ ---
# Converter para array denso só para visualizarmos (não faça isso com imagens grandes!)
print("\nA Matriz Laplaciana A construída:")
print(A.toarray())

# Visualizar a estrutura de esparsidade (Spy Plot)
plt.figure(figsize=(5, 5))
plt.spy(A, markersize=10, color='b')
plt.title(f"Estrutura da Matriz A ({n_pixels}x{n_pixels})")
plt.grid(True)
plt.show()